In [1]:
import torch
from torchvision import datasets
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
train_data = datasets.MNIST(
    root = 'data',
    train = True,
    transform = ToTensor(),
    download = True
)

test_data = datasets.MNIST(
    root = 'data',
    train = False,
    transform = ToTensor(),
    download = True
)

In [3]:
train_data

Dataset MNIST
    Number of datapoints: 60000
    Root location: data
    Split: Train
    StandardTransform
Transform: ToTensor()

In [4]:
test_data

Dataset MNIST
    Number of datapoints: 10000
    Root location: data
    Split: Test
    StandardTransform
Transform: ToTensor()

In [5]:
train_data.data.shape

torch.Size([60000, 28, 28])

In [6]:
test_data.data.shape

torch.Size([10000, 28, 28])

In [7]:
train_data.targets

tensor([5, 0, 4,  ..., 5, 6, 8])

In [8]:
loaders = {
    'train' : DataLoader(train_data,
                         batch_size = 100,
                         shuffle = True,
                         num_workers = 1),

    'test' : DataLoader(test_data,
                         batch_size = 100,
                         shuffle = True,
                         num_workers = 1)                                           
}

In [9]:
loaders

{'train': <torch.utils.data.dataloader.DataLoader at 0x16969a9d0>,
 'test': <torch.utils.data.dataloader.DataLoader at 0x169691310>}

In [10]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size = 5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size = 5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training = self.training)
        x = self.fc2(x)

        return F.softmax(x)    

In [11]:
model = CNN()

In [12]:
optimizer = optim.Adam(model.parameters(), lr = 0.01)
loss_fn = nn.CrossEntropyLoss()

In [13]:
def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(loaders['train']):
        optimizer.zero_grad()
        output = model(data)
        loss = loss_fn(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 20 == 0:
            print(f'train epoch: {epoch} [{batch_idx * len(data)} / {len(loaders["train"].dataset)} ({100. * batch_idx/len(loaders["train"]):.0f}%)]\t{loss.item():.6f}')

def test():
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in loaders['test']:
            output = model(data)
            test_loss = loss_fn(output, target).item()
            pred = output.argmax(dim = 1, keepdim = True)
            correct += pred.eq(target.view_as(pred)).sum().item()
    test_loss /= len(loaders['test'].dataset)
    print(f'\ntest set: average loss: {test_loss:.4f}, accuracy {correct}/{len(loaders["test"].dataset)} ({100. * correct / len(loaders["test"].dataset):.0f}%\n)')      

In [14]:
for epoch in range(1, 11):
    train(epoch)
    test()

/var/folders/31/88846tc925lfks7qc_y5bbmr0000gn/T/ipykernel_8452/2304616029.py:18: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.softmax(x)


train epoch: 1 [0 / 60000 (0%)]	2.302910
train epoch: 1 [2000 / 60000 (3%)]	2.088881
train epoch: 1 [4000 / 60000 (7%)]	1.916017
train epoch: 1 [6000 / 60000 (10%)]	1.873755
train epoch: 1 [8000 / 60000 (13%)]	1.798738
train epoch: 1 [10000 / 60000 (17%)]	1.822402
train epoch: 1 [12000 / 60000 (20%)]	1.820205
train epoch: 1 [14000 / 60000 (23%)]	1.785162
train epoch: 1 [16000 / 60000 (27%)]	1.722506
train epoch: 1 [18000 / 60000 (30%)]	1.823986
train epoch: 1 [20000 / 60000 (33%)]	1.786101
train epoch: 1 [22000 / 60000 (37%)]	1.705844
train epoch: 1 [24000 / 60000 (40%)]	1.675908
train epoch: 1 [26000 / 60000 (43%)]	1.664334
train epoch: 1 [28000 / 60000 (47%)]	1.684505
train epoch: 1 [30000 / 60000 (50%)]	1.636376
train epoch: 1 [32000 / 60000 (53%)]	1.682063
train epoch: 1 [34000 / 60000 (57%)]	1.649948
train epoch: 1 [36000 / 60000 (60%)]	1.620918
train epoch: 1 [38000 / 60000 (63%)]	1.631848
train epoch: 1 [40000 / 60000 (67%)]	1.701685
train epoch: 1 [42000 / 60000 (70%)]	1.668206